In [ ]:
#動防所財抽
#營業登記
import pandas as pd
import numpy as np
pd.options.display.max_colwidth=0 #顯示全部字串
#營業登記檔
bgm = pd.read_csv(r'c:\ais\\bgmopen1\BGMOPEN1.csv')
bgm = bgm.fillna(0)
bgm['統一編號']=bgm['統一編號'].astype('int')
bgm['總機構統一編號']=bgm['總機構統一編號'].astype('int')
bgm['設立日期']=bgm['設立日期'].astype('int')
bgm['行業代號']=bgm['行業代號'].astype('int')
bgm['行業代號1']=bgm['行業代號1'].astype('int')
bgm['行業代號2']=bgm['行業代號2'].astype('int')
bgm['行業代號3']=bgm['行業代號3'].astype('int')
bgm.dtypes
#預算明細分類帳
b = pd.read_csv(r'D:\gdrive\動防所108期中財抽\106_10807CBA\資料匯出-20190920083721-預算明細分類帳.csv'
               ,dtype={'金額':int})




In [97]:
#農藥販賣業者核發情形
s1 = pd.read_excel(r'D:\gdrive\動防所108期中財抽\2.1.農藥販賣業執照核發情形\農藥販賣業執照批發.xls'
                  ,header=2
                  )
s1['類別']="批發"
s2=pd.read_excel(r'D:\gdrive\動防所108期中財抽\2.1.農藥販賣業執照核發情形\農藥販賣業執照零售.xls'
                  ,header=2
                  )
s2=s2[~s2['執照號碼'].isin(s1['執照號碼'])] #去除批發業者
s2['類別']="零售"

s=pd.concat([s1,s2],axis=0)
s.reset_index(drop=True, inplace=True)
#匯出地址
#s['商號地址'].to_csv("store_addr.csv", encoding='cp950')
#s[s['倉儲地址'].str.len()>6]['倉儲地址'].to_csv("warhouse_addr.csv", encoding='cp950')

#倉儲地址座標化
a1 = pd.read_csv(r'D:\gdrive\動防所108期中財抽\2.1.農藥販賣業執照核發情形\store_addr_info.csv')
a2 = pd.read_csv(r'D:\gdrive\動防所108期中財抽\2.1.農藥販賣業執照核發情形\warehouse_addr_info.csv')
a1 = a1.drop('id',axis=1)
a2 = a2.drop('id',axis=1)
s=s.merge(a1, how='left', left_on='商號地址', right_on='Address')
s=s.merge(a2, how='left', left_on='倉儲地址', right_on='Address', suffixes=("_s", "_w"))

#批發業者地址無效或未詳實填載者
s[ (s['類別']=='批發')
 & ( (s['倉儲地址'].str.len()<6)
   | (s['倉儲地址'].isna())
   | (s['Response_Address_w'].str.contains("找不到"))
   )
 ][['執照號碼', '商號名稱', '分支機構', '商號統編', '商號電話', '負責人', '管理人員', '商號地址', '倉儲地址',
   '狀態', '執照發證日期', '停業/歇業/撤銷日期', '類別'
  ]].to_csv("ws_with_invalid_warhouse_address.csv")
#批發業者之營業地點或倉儲地點位於住宅區者
s[ (s['類別']=='批發')
 & ((s['分區_s']=='住宅區')
   |(s['分區_w']=='住宅區')
   )   
 ][['執照號碼', '商號名稱', '商號統編', '商號電話', '負責人', '管理人員', '商號地址', '倉儲地址',
   '狀態', '執照發證日期', '類別', 
   'Response_Address_s','分區_s', '都市計畫_s',
   'Response_Address_w','分區_w', '都市計畫_w'
  ]]


,執照號碼,商號名稱,商號統編,商號電話,負責人,管理人員,商號地址,倉儲地址,狀態,執照發證日期,類別,Response_Address_s,分區_s,都市計畫_s,Response_Address_w,分區_w,都市計畫_w
7,花蓮縣 O00020,豐源農藥行,94622810,8762235,黃宏文,李維禎,花蓮縣鳳林鎮鳳仁里中正路２段２２９號,花蓮縣鳳林鎮中正路2段227號,核准,民國105年10月26日,批發,花蓮縣鳳林鎮鳳仁里13鄰中正路二段229號,住宅區,鳳林都市計畫,花蓮縣鳳林鎮鳳仁里13鄰中正路二段227號,住宅區,鳳林都市計畫
14,花蓮縣 O00050,呈豐企業有限公司,94790787,038538588,林次郎,林次郎,花蓮縣吉安鄉南昌村南昌路152號1樓,花蓮縣吉安鄉南昌村南昌路152號1樓,核准,民國105年10月28日,批發,花蓮縣吉安鄉南昌村7鄰南昌路152號,住宅區,變更吉安都市計畫(第四次通盤檢討)(第一階段),花蓮縣吉安鄉南昌村7鄰南昌路152號,住宅區,變更吉安都市計畫(第四次通盤檢討)(第一階段)
15,花蓮縣 O00051,毅欣農化有限公司,94844854,038538568,林孟德,林孟德,花蓮縣吉安鄉南昌村南昌路154號,花蓮縣吉安鄉南昌路154號,核准,民國105年10月28日,批發,花蓮縣吉安鄉南昌村7鄰南昌路154號,住宅區,變更吉安都市計畫(第四次通盤檢討)(第一階段),花蓮縣吉安鄉南昌村7鄰南昌路154號,住宅區,變更吉安都市計畫(第四次通盤檢討)(第一階段)
16,花蓮縣 O00054,金峰行,94789732,038573163,李梅珠,李梅珠,花蓮縣花蓮市國富里中央路三段658號,花蓮縣花蓮市國富里新生南路247號,核准,民國105年10月28日,批發,花蓮縣花蓮市國富里34鄰中央路三段658號,住宅區,變更吉安都市計畫(第四次通盤檢討)(第一階段),找不到指定的門牌地址。,NaN,NaN


In [87]:
#匯出有座標的業者
s.dropna(subset=['Response_X']).to_csv("pesticide_seller.csv")

KeyError: ['Response_X']

In [ ]:
#匯出有效的倉儲地址
s[s['倉儲地址'].str.len()>6]['倉儲地址']



In [ ]:
bgm[bgm['營業地址'].str.contains("花蓮縣")
   &(bgm['營業人名稱'].str.contains("農藥")
    |bgm['名稱'].str.contains("農藥")
    |bgm['名稱1'].str.contains("農藥")
    |bgm['名稱2'].str.contains("農藥")
    |bgm['名稱3'].str.contains("農藥") 
    )
   ]

In [ ]:
url = "http://data.coa.gov.tw/Service/OpenData/FromM/AgricultureiResidueVegetable.aspx"
bad = pd.read_json(url)
bad.columns

In [ ]:
bad[bad["採樣地點"].str.contains("花蓮")]

In [ ]:
#似未辦營登之農藥行 seller not in bgm 
#排除農會
s['商號名稱'] = s['商號名稱'].str.replace('新峰', '新峯')
s['商號名稱'] = s['商號名稱'].str.replace('惠民農藥行', '恵民農藥行')
s['商號名稱'] = s['商號名稱'].str.replace('廣益農葯行', '廣益農藥行')


sib = s.merge(bgm[bgm['營業地址'].str.contains("花蓮縣")],how="left", left_on='商號名稱', right_on='營業人名稱')
sib[sib['營業人名稱'].isnull()
   & (sib['商號統編'].str.contains('\*'))
   & (~sib['商號名稱'].str.contains('農會'))   
   ]


In [ ]:
bgm.columns

In [ ]:
s